[View in Colaboratory](https://colab.research.google.com/github/vijaykyr/tensorflow_teaching_examples/blob/dev/review_project/03_Models.ipynb)

In [1]:
import shutil, os
import numpy as np
import pandas as pd
import tensorflow as tf
pd.set_option('display.max_columns', 100)

print(tf.__version__)

1.10.0


In [0]:
SEED=1 # for reproducibility

### Colab initialization

Remove this before publishing

In [3]:
!pip install --upgrade tensorflow==1.10 # keras estimator breaks in tf 1.11, global step never increments

Requirement already up-to-date: tensorflow==1.10 in /usr/local/lib/python2.7/dist-packages (1.10.0)


In [0]:
from google.colab import auth
auth.authenticate_user()

In [5]:
!gcloud config set project vijays-sandbox

Updated property [core/project].


### Start from engineered dataset

Already includes engineered features and labels

In [6]:
!gsutil cp gs://cloud-training-demos/courses/machine_learning/asl_review_project/data.csv data.csv

Copying gs://cloud-training-demos/courses/machine_learning/asl_review_project/data.csv...
| [1 files][325.3 MiB/325.3 MiB]                                                
Operation completed over 1 objects/325.3 MiB.                                    


**Explore Data**

In [7]:
dfXy = pd.read_csv('data.csv')
dfXy.head()

,surprise,reported_EPS,consensus_EPS,symbol,datetime,events,file,text,release_hr,release_minute,time_of_release,date,label,price_change,close_m_open,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,close_MAX_prior_20_days,close_MAX_prior_260_days,close_AVG_prior_5_days,close_AVG_prior_20_days,close_AVG_prior_260_days,close_STDDEV_prior_5_days,close_STDDEV_prior_20_days,close_STDDEV_prior_260_days,close_values_prior_260
0,0.00,-0.15,-0.15,A,2003-05-19 16:23:06,regulation_fd_disclosure,A/A-8K-20030519162306.txt.gz\n,\nITEM: Regulation FD Disclosure\nTable of Con...,16,23,after,2003-05-19,UP,0.034816,-0.59,1.047066,0.985171,0.699549,1.077369,1.077369,2.001934,1.058027,1.036718,1.115747,0.012708,0.021184,0.300234,"[27.81, 30.04, 28.74, 27.95, 28.46, 31.05, 29...."
1,71.43,-0.02,-0.07,A,2003-08-18 16:30:26,NaN,A/A-8K-20030818163026.txt.gz\n,\nITEM: \nTable of Contents\nTable of Contents...,16,30,after,2003-08-18,UP,0.035619,0.68,0.935886,0.913179,0.483081,0.969724,0.978183,0.989760,0.955476,0.951380,0.725843,0.014615,0.021125,0.131667,"[16.27, 16.87, 17.2, 16.72, 16.46, 15.89, 16.0..."
2,-4.55,0.21,0.22,A,2004-02-17 16:19:29,NaN,A/A-8K-20040217161929.txt.gz\n,\nITEM: \n \n \n \n \n \n \n \n \n \n \n \n \n...,16,19,after,2004-02-17,DOWN,-0.039744,0.41,0.984796,0.891971,0.305415,0.997599,0.997599,0.997599,0.991571,0.953681,0.587837,0.005472,0.034869,0.174599,"[16.3, 16.32, 12.26, 11.45, 11.73, 12.05, 12.4..."
3,0.00,0.24,0.24,A,2004-05-17 16:25:01,NaN,A/A-8K-20040517162501.txt.gz\n,\nITEM: \n \n \n \n \n \n \n \n \n \n \n \n \n...,16,25,after,2004-05-17,UP,0.026646,0.07,0.995690,0.995690,0.607759,1.026646,1.191223,1.469044,1.013950,1.079330,1.023271,0.013927,0.058654,0.216846,"[16.07, 15.99, 16.02, 16.33, 16.3, 16.24, 16.2..."
4,7.14,0.30,0.28,A,2004-08-12 16:53:00,results_of_operations_and_financial_condition,A/A-8K-20040812165300.txt.gz\n,\nITEM: Results of Operations and Financial Co...,16,53,after,2004-08-12,UP,0.086382,-2.10,1.106199,1.106199,1.045732,1.161585,1.313008,1.904980,1.128354,1.201347,1.400917,0.024174,0.063852,0.210281,"[21.73, 21.55, 21.56, 20.83, 20.58, 20.67, 20...."


In [8]:
dfXy = dfXy.dropna() # remove rows with null values
dfXy.describe()

,surprise,reported_EPS,consensus_EPS,release_hr,release_minute,price_change,close_m_open,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,close_MAX_prior_20_days,close_MAX_prior_260_days,close_AVG_prior_5_days,close_AVG_prior_20_days,close_AVG_prior_260_days,close_STDDEV_prior_5_days,close_STDDEV_prior_20_days,close_STDDEV_prior_260_days
count,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000,14351.000000
mean,2.168222,0.728475,0.717198,11.369870,24.464567,0.001728,0.015911,0.977410,0.941194,0.752378,1.021649,1.058376,1.332590,0.999480,0.999513,1.035995,0.018126,0.034813,0.149644
std,369.845604,2.559523,0.752646,4.283074,16.968143,0.043719,2.030058,0.052699,0.073846,0.165629,0.070620,0.125448,1.243288,0.054224,0.084827,0.540218,0.027083,0.041305,0.415959
min,-38300.000000,-283.400000,-16.980000,6.000000,0.000000,-0.998461,-60.540000,0.000303,0.000303,0.000276,0.716792,0.820030,0.836735,0.595825,0.522218,0.138430,0.000718,0.003070,0.019525
25%,0.000000,0.350000,0.340000,8.000000,10.000000,-0.016221,-0.570000,0.954355,0.910638,0.664773,0.992259,1.004405,1.042547,0.974775,0.961044,0.877483,0.008697,0.016786,0.067255
50%,3.600000,0.610000,0.590000,9.000000,22.000000,0.000692,-0.010000,0.980378,0.951072,0.775554,1.013953,1.031660,1.129253,0.997408,0.992013,0.962872,0.013611,0.025127,0.096834
75%,10.770000,1.000000,0.960000,16.000000,38.000000,0.020236,0.550000,1.003014,0.982435,0.869574,1.040982,1.075391,1.345239,1.020761,1.027671,1.084247,0.021517,0.039368,0.153940
max,7900.000000,13.870000,10.650000,21.000000,59.000000,0.430416,41.000000,1.601758,1.598956,1.400000,4.968535,5.172387,124.404762,2.559268,4.416486,48.571245,2.168584,1.484452,43.286283


### DNN Using Only Numeric Features

47.64%

In [9]:
#Select features
ix_price_features = dfXy.columns.str.contains('close_')
price_features = dfXy.columns[ix_price_features].tolist()
price_features.remove('close_values_prior_260') # For RNN, which is a list. We don't want.
FEATURES = price_features + ['surprise'] 
FEATURES

['close_m_open',
 'close_MIN_prior_5_days',
 'close_MIN_prior_20_days',
 'close_MIN_prior_260_days',
 'close_MAX_prior_5_days',
 'close_MAX_prior_20_days',
 'close_MAX_prior_260_days',
 'close_AVG_prior_5_days',
 'close_AVG_prior_20_days',
 'close_AVG_prior_260_days',
 'close_STDDEV_prior_5_days',
 'close_STDDEV_prior_20_days',
 'close_STDDEV_prior_260_days',
 'surprise']

In [0]:
#Create Train/Eval Datasets
np.random.seed(0) # this is the only seed that isn't parameterized because the data split should never change
msk = np.random.rand(len(dfXy)) < 0.8 
traindf = dfXy[msk].copy()
evaldf = dfXy[~msk].copy()


In [0]:
#Convert string labels to numeric
traindf['label_numeric'] = traindf.label.apply(lambda x: {'DOWN': 0, 'STAY': 1, 'UP':2}[x])
evaldf['label_numeric'] = evaldf.label.apply(lambda x: {'DOWN': 0, 'STAY': 1, 'UP':2}[x])

In [12]:
###Hyperparameters
dropout_rate=0
units=10
input_shape = [len(FEATURES)]
num_classes=3
loss = 'sparse_categorical_crossentropy'
learning_rate=1e-3
epochs=10
batch_size=128

from tensorflow import keras
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense, Dropout, Embedding
from tensorflow.python.keras import backend as K

np.random.seed(SEED)
tf.set_random_seed(SEED) 

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

model = models.Sequential()
model.add(Dense(units=units, input_shape=input_shape, activation='relu', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED)))
model.add(Dropout(rate=dropout_rate,seed=SEED))
model.add(Dense(units=num_classes, activation='softmax', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED)))

optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=3)]

history = model.fit(
        traindf[FEATURES],
        traindf.label_numeric,
        callbacks=callbacks,
        epochs=epochs,
        validation_data=(evaldf[FEATURES], evaldf.label_numeric),
        verbose=2,  # Logs once per epoch.
        batch_size=batch_size)

Train on 11530 samples, validate on 2821 samples
Epoch 1/10
 - 0s - loss: 1.3782 - acc: 0.3876 - val_loss: 1.1623 - val_acc: 0.4201
Epoch 2/10
 - 0s - loss: 1.0864 - acc: 0.4554 - val_loss: 1.0630 - val_acc: 0.4608
Epoch 3/10
 - 0s - loss: 1.0705 - acc: 0.4641 - val_loss: 1.0793 - val_acc: 0.4566
Epoch 4/10
 - 0s - loss: 1.0539 - acc: 0.4656 - val_loss: 1.0794 - val_acc: 0.4583
Epoch 5/10
 - 0s - loss: 1.0591 - acc: 0.4686 - val_loss: 1.0534 - val_acc: 0.4700
Epoch 6/10
 - 0s - loss: 1.0478 - acc: 0.4698 - val_loss: 1.0392 - val_acc: 0.4725
Epoch 7/10
 - 0s - loss: 1.0457 - acc: 0.4711 - val_loss: 1.0384 - val_acc: 0.4722
Epoch 8/10
 - 0s - loss: 1.0571 - acc: 0.4671 - val_loss: 1.0459 - val_acc: 0.4743
Epoch 9/10
 - 0s - loss: 1.0415 - acc: 0.4721 - val_loss: 1.0443 - val_acc: 0.4754
Epoch 10/10
 - 0s - loss: 1.0471 - acc: 0.4717 - val_loss: 1.0615 - val_acc: 0.4764


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                150       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 33        
Total params: 183
Trainable params: 183
Non-trainable params: 0
_________________________________________________________________


### Create bag of words representation of 8Ks

Takes 2min to run

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# Select top 'k' of the vectorized features.
TOP_K = 2000 
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 1)
# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 5

vectorizer = CountVectorizer(
    ngram_range=NGRAM_RANGE,
    strip_accents='unicode',
    stop_words='english',
    min_df=MIN_DOCUMENT_FREQUENCY,
    ) # returns sklearn.feature_extraction.text.CountVectorizer

x_train_text = vectorizer.fit_transform(traindf.text) #takes ~2min, returns scipy.sparse.csr.csr_matrix
x_val_text = vectorizer.transform(evaldf.text)
print('{} tokens identified'.format(len(vectorizer.get_feature_names())))

28171 tokens identified


**Select 2000 most important tokens using f_classif**

Score each of the 28171 tokens based on their predictiveness of the label. Keep only the 2000 highest scoring tokens

In [15]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=min(TOP_K, x_train_text.shape[1]))
selector.fit(x_train_text, traindf.label_numeric)
print('x_train_text shape before feature selection: {}'.format(x_train_text.shape))
x_train_text = selector.transform(x_train_text)
x_val_text = selector.transform(x_val_text)
print('x_train_text shape after feature selection: {}'.format(x_train_text.shape))

x_train_text shape before feature selection: (11530, 28171)
x_train_text shape after feature selection: (11530, 2000)


**Display Tokens**

Sorted to show highest scoring first

In [16]:
feature_names = vectorizer.get_feature_names()
token_list = []
for i in selector.get_support(indices=True):
    token_list.append((round(selector.scores_[i],2),feature_names[i]))

sorted(token_list, reverse=True)

[(81.49, u'electric'),
 (64.73, u'weather'),
 (64.08, u'energy'),
 (42.12, u'trademarks'),
 (41.44, u'utility'),
 (40.19, u'regulated'),
 (38.28, u'earnings'),
 (35.73, u'generation'),
 (34.58, u'utilities'),
 (33.36, u'transmission'),
 (30.62, u'fiscal'),
 (30.22, u'megawatts'),
 (28.86, u'maintenance'),
 (26.4, u'competitors'),
 (26.07, u'2003'),
 (25.87, u'gaap'),
 (25.53, u'generating'),
 (25.3, u'cents'),
 (24.37, u'2004'),
 (23.81, u'regulatory'),
 (23.76, u'normal'),
 (23.43, u'electricity'),
 (23.13, u'communicating'),
 (22.53, u'summary'),
 (22.4, u'unregulated'),
 (22.37, u'plant'),
 (21.96, u'territory'),
 (21.64, u'mechanisms'),
 (21.54, u'projects'),
 (21.16, u'calif'),
 (20.79, u'afudc'),
 (20.65, u'2008'),
 (20.62, u'annually'),
 (20.58, u'decommissioning'),
 (20.48, u'press'),
 (20.29, u'outage'),
 (20.04, u'wisconsin'),
 (19.68, u'profile'),
 (19.57, u'warmer'),
 (19.41, u'generator'),
 (18.87, u'megawatt'),
 (18.75, u'outages'),
 (18.61, u'power'),
 (18.13, u'technolo

### DNN Using Numeric + Text Features

49.17% 

This model uses a naive approach of concatenating all 2000 text features to the 13 numeric features and feeding them all at once to the DNN

In [0]:
x_train = np.hstack([x_train_text.toarray(),traindf[FEATURES]])
x_val = np.hstack([x_val_text.toarray(),evaldf[FEATURES]])

In [18]:
#Hyperparameters
dropout_rate=0.5
units=64
input_shape = x_train.shape[1:]
num_classes=3
loss = 'sparse_categorical_crossentropy'
learning_rate=1e-3
epochs=10
batch_size=128

from tensorflow import keras
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense, Dropout, Embedding
from tensorflow.python.keras import backend as K

np.random.seed(SEED)
tf.set_random_seed(SEED) 

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

model = models.Sequential()
model.add(Dense(units=units, input_shape=input_shape, activation='relu', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED)))
model.add(Dropout(rate=dropout_rate,seed=SEED))
model.add(Dense(units=num_classes, activation='softmax', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED)))

optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=3)]

history = model.fit(
        x_train,
        traindf.label_numeric,
        callbacks=callbacks,
        epochs=epochs,
        validation_data=(x_val, evaldf.label_numeric),
        verbose=2,  # Logs once per epoch.
        batch_size=batch_size)

Train on 11530 samples, validate on 2821 samples
Epoch 1/10
 - 1s - loss: 1.3263 - acc: 0.3964 - val_loss: 1.1270 - val_acc: 0.4612
Epoch 2/10
 - 1s - loss: 1.1249 - acc: 0.4435 - val_loss: 1.1127 - val_acc: 0.4573
Epoch 3/10
 - 1s - loss: 1.0918 - acc: 0.4668 - val_loss: 1.0794 - val_acc: 0.4810
Epoch 4/10
 - 1s - loss: 1.0749 - acc: 0.4723 - val_loss: 1.0914 - val_acc: 0.4800
Epoch 5/10
 - 1s - loss: 1.0496 - acc: 0.4919 - val_loss: 1.0545 - val_acc: 0.4832
Epoch 6/10
 - 1s - loss: 1.0350 - acc: 0.4997 - val_loss: 1.0732 - val_acc: 0.4747
Epoch 7/10
 - 1s - loss: 1.0296 - acc: 0.5088 - val_loss: 1.0711 - val_acc: 0.4828
Epoch 8/10
 - 1s - loss: 1.0138 - acc: 0.5210 - val_loss: 1.0600 - val_acc: 0.4917


In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 64)                128960    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 195       
Total params: 129,155
Trainable params: 129,155
Non-trainable params: 0
_________________________________________________________________


### With Embedding

52.14% (after 5 epochs)

If we take a more nuanced approach of first learning an 50 dimensional embedding of the 2000 text features, then concatenating the embedded representation with the numeric features, we get better performance

In [20]:
#Hyperparameters
EMBEDDING_DIM = 50
dropout_rate=0.5
units=64
input_shape = x_train.shape[1:]
num_classes=3
loss = 'sparse_categorical_crossentropy'
learning_rate=1e-3
epochs=10
batch_size=128

from tensorflow import keras
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense, Dropout, Embedding
from tensorflow.python.keras import backend as K

np.random.seed(SEED)
tf.set_random_seed(SEED) 

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

# using keras functional API instead of sequential API to insert surprise upstream of embedding.
text_input = keras.Input(shape=x_train_text.shape[1:], dtype='float32', name='text_input')
text_embedded = Dense(units=EMBEDDING_DIM, activation='relu', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED))(text_input)
auxiliary_input = keras.Input(shape=(len(FEATURES),), name='auxiliary_input')
x = keras.layers.concatenate([text_embedded, auxiliary_input])
x = Dense(units=units, activation='relu', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED))(x)
x = Dropout(rate=dropout_rate, seed=SEED)(x)
output = Dense(units=num_classes, activation='softmax', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED))(x)
model = keras.Model(inputs=[text_input,auxiliary_input], outputs=output)

optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

history = model.fit(
        {'text_input':x_train_text, 'auxiliary_input':traindf[FEATURES]},
        traindf.label_numeric,
        epochs=epochs,
        validation_data=(
            {'text_input':x_val_text, 'auxiliary_input':evaldf[FEATURES]}, 
            evaldf.label_numeric),
        verbose=2,  # Logs once per epoch.
        batch_size=batch_size)

Train on 11530 samples, validate on 2821 samples
Epoch 1/10
 - 1s - loss: 1.6352 - acc: 0.4127 - val_loss: 1.2850 - val_acc: 0.4860
Epoch 2/10
 - 1s - loss: 1.3073 - acc: 0.4421 - val_loss: 1.2334 - val_acc: 0.4821
Epoch 3/10
 - 1s - loss: 1.2276 - acc: 0.4552 - val_loss: 1.1680 - val_acc: 0.5055
Epoch 4/10
 - 1s - loss: 1.1736 - acc: 0.4753 - val_loss: 1.1531 - val_acc: 0.5055
Epoch 5/10
 - 1s - loss: 1.1524 - acc: 0.4925 - val_loss: 1.1309 - val_acc: 0.5214
Epoch 6/10
 - 1s - loss: 1.1222 - acc: 0.4958 - val_loss: 1.1166 - val_acc: 0.5126
Epoch 7/10
 - 1s - loss: 1.1114 - acc: 0.4996 - val_loss: 1.1202 - val_acc: 0.5133
Epoch 8/10
 - 1s - loss: 1.0861 - acc: 0.5091 - val_loss: 1.0996 - val_acc: 0.5105
Epoch 9/10
 - 1s - loss: 1.0709 - acc: 0.5109 - val_loss: 1.0941 - val_acc: 0.5108
Epoch 10/10
 - 1s - loss: 1.0686 - acc: 0.5214 - val_loss: 1.0864 - val_acc: 0.4981


In [21]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         (None, 2000)         0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 50)           100050      text_input[0][0]                 
__________________________________________________________________________________________________
auxiliary_input (InputLayer)    (None, 14)           0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 64)           0           dense_4[0][0]                    
                                                                 auxiliary_input[0][0]            
__________

### Package as Estimator

This allows for distributed training and easy deployment of REST API.

We create a task.py to parse command line parameters and a model.py for the remaining code.

Note here we remove the single-threaded constraint which allows faster training but no longer guarantees reproducibility

In [22]:
%%bash
mkdir stockprediction
touch stockprediction/__init__.py

mkdir: cannot create directory ‘stockprediction’: File exists


In [23]:
%%writefile stockprediction/task.py

import argparse
from . import model

if __name__ == '__main__':
    # parse command line arguments
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--output_dir',
        help='location to write checkpoints and export models',
        required=True
    )
    parser.add_argument(
        '--data_path',
        help='can be a local path or a GCS url (gs://...)',
        required=True
    )

    args, _ = parser.parse_known_args()
    hparams = args.__dict__
    
    model.train_and_evaluate(hparams)

Overwriting stockprediction/task.py


In [24]:
%%writefile stockprediction/model.py

import shutil
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, f_classif

from tensorflow import keras
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense, Dropout, Embedding
from tensorflow.python.keras import backend as K

tf.logging.set_verbosity('INFO')

### 0. Hyperparameters
LEARNING_RATE=1e-3 # .001
EPOCHS=10
BATCH_SIZE=128
SEED=1
EMBEDDING_DIM = 50
DROPOUT_RATE=0.5
UNITS=64 # units in each dense layer
NUM_CLASSES=3
TOP_K = 2000 # Select top 'k' of the vectorized features.
NGRAM_RANGE = (1, 1) # Range (inclusive) of n-gram sizes for tokenizing text.
MIN_DOCUMENT_FREQUENCY = 5 # Minimum document/corpus frequency below which a token will be discarded.

np.random.seed(SEED)
tf.set_random_seed(SEED) 

### 1. Load Data
def load_data(data_path):
  dfXy = pd.read_csv(data_path)
  dfXy = dfXy.dropna() # remove rows with null values

  #Create Train/Eval Datasets
  np.random.seed(0) # this is the only seed that isn't parameterized because the data split should never change
  msk = np.random.rand(len(dfXy)) < 0.8 
  traindf = dfXy[msk].copy()
  evaldf = dfXy[~msk].copy()

  #Convert string labels to numeric
  traindf['label_numeric'] = traindf.label.apply(lambda x: {'DOWN': 0, 'STAY': 1, 'UP':2}[x])
  evaldf['label_numeric'] = evaldf.label.apply(lambda x: {'DOWN': 0, 'STAY': 1, 'UP':2}[x])

  return traindf, evaldf

### 2. Vectorize text
def vectorize_text(traindf, evaldf):
  vectorizer = CountVectorizer(
      ngram_range=NGRAM_RANGE,
      strip_accents='unicode',
      stop_words='english',
      min_df=MIN_DOCUMENT_FREQUENCY,
      ) # returns sklearn.feature_extraction.text.CountVectorizer

  x_train_text = vectorizer.fit_transform(traindf.text) #takes ~2min, returns scipy.sparse.csr.csr_matrix
  x_val_text = vectorizer.transform(evaldf.text)

  selector = SelectKBest(f_classif, k=min(TOP_K, x_train_text.shape[1]))
  selector.fit(x_train_text, traindf.label_numeric)
  x_train_text = selector.transform(x_train_text)
  x_val_text = selector.transform(x_val_text)
  return x_train_text, x_val_text

### 3. Model Code
def build_model(num_numeric_features,num_text_features):
  # using keras functional API instead of sequential API to insert surprise upstream of embedding.
  text_input = keras.Input(shape=num_text_features, dtype='float32', name='text_input')
  text_embedded = Dense(units=EMBEDDING_DIM, activation='relu', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED))(text_input)
  auxiliary_input = keras.Input(shape=(num_numeric_features,), name='auxiliary_input')
  x = keras.layers.concatenate([text_embedded, auxiliary_input])
  x = Dense(units=UNITS, activation='relu', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED))(x)
  x = Dropout(rate=DROPOUT_RATE, seed=SEED)(x)
  output = Dense(units=NUM_CLASSES, activation='softmax', kernel_initializer=tf.glorot_uniform_initializer(seed=SEED))(x)
  model = keras.Model(inputs=[text_input,auxiliary_input], outputs=output)

  optimizer = tf.keras.optimizers.Adam(lr=LEARNING_RATE)
  model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['acc'])

  return model

### 4. Input Function

def input_fn(features, labels, mode):
  dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    
  if mode == tf.estimator.ModeKeys.TRAIN:
    dataset = dataset.shuffle(50000,seed=SEED).repeat(-1)

  # Shuffle, repeat, and batch the examples.
  return dataset.batch(BATCH_SIZE)


def train_and_evaluate(hparams):
  ### 1. Load Data
  traindf, evaldf = load_data(hparams['data_path'])
  ### 2. Vectorize text
  x_train_text, x_val_text = vectorize_text(traindf, evaldf)

  # Select Numeric Features
  ix_price_features = traindf.columns.str.contains('close_')
  price_features = traindf.columns[ix_price_features].tolist()
  price_features.remove('close_values_prior_260') # For RNN, which is a list. We don't want.
  FEATURES = price_features + ['surprise'] 
 
  ### 3. Build Estimator
  model = build_model(
    num_numeric_features=len(FEATURES),
    num_text_features=x_train_text.shape[1:])
  run_config = tf.estimator.RunConfig(save_checkpoints_steps=100, tf_random_seed=SEED)
  estimator = tf.keras.estimator.model_to_estimator(keras_model=model, model_dir=hparams['output_dir'], config=run_config)

  # Create TrainSpec
  train_steps = EPOCHS * len(traindf) / BATCH_SIZE
  train_spec = tf.estimator.TrainSpec(
      input_fn=lambda: input_fn(
          {'text_input':x_train_text.toarray(), 'auxiliary_input':traindf[FEATURES]},
          traindf.label_numeric,
          mode=tf.estimator.ModeKeys.TRAIN),
      max_steps=train_steps
  )

  # Create EvalSpec
  exporter = None #tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec = tf.estimator.EvalSpec(
      input_fn=lambda: input_fn(
          {'text_input':x_val_text.toarray(), 'auxiliary_input':evaldf[FEATURES]},
          evaldf.label_numeric,
          mode=tf.estimator.ModeKeys.EVAL),
      steps=None,
      exporters=exporter,
      start_delay_secs=10,
      throttle_secs=1
  )

  # Start training
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Overwriting stockprediction/model.py


### Test Package Locally

In [25]:
%%bash
OUTDIR='model_trained'
rm -rf $OUTDIR
gcloud ml-engine local train \
   --module-name=stockprediction.task \
   --package-path=${PWD}/stockprediction \
   -- \
   --output_dir=$OUTDIR \
   --data_path='data.csv'

INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--output_dir=model_trained', u'--data_path=data.csv'], u'job_name': u'stockprediction.task'}, u'task': {}}
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7096b75dd0>, '_evaluation_master': '', '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': 1, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'model_trained', '_train_distribute': None, '_save_summary_steps': 100}
2018-10-10 19:40:52.436816: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU support

### Train on CMLE

Here we use a GCS hosted data.csv as our 'data_path' so that our CMLE service account can access it from the cloud. 

We use the https:// prefix as opposed to gs:// so that the pandas.read_csv() method can access it. 

Alternatively we could have used the GCS python client to first download the file to the cloud machine and then read as a local file using pandas.read_csv()

In [26]:
%%bash
TFVERSION='1.10'
BUCKET='vijays-sandbox'
OUTDIR=gs://${BUCKET}/stockprediction/trained/
JOBNAME=stockprediction_$(date -u +%y%m%d_%H%M%S)
REGION=us-central1
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
 --region=$REGION \
 --module-name=stockprediction.task \
 --package-path=${PWD}/stockprediction \
 --job-dir=$OUTDIR \
 --scale-tier=BASIC_GPU \
 --runtime-version=$TFVERSION \
 -- \
 --output_dir=$OUTDIR \
 --data_path='https://storage.googleapis.com/cloud-training-demos/courses/machine_learning/asl_review_project/data.csv'

jobId: stockprediction_181010_194243
state: QUEUED


Removing gs://vijays-sandbox/stockprediction/trained/#1539199732481688...
Removing gs://vijays-sandbox/stockprediction/trained/graph.pbtxt#1539199580021405...
Removing gs://vijays-sandbox/stockprediction/trained/checkpoint#1539199734505208...
Removing gs://vijays-sandbox/stockprediction/trained/model.ckpt-500.data-00000-of-00001#1539199664337415...
Removing gs://vijays-sandbox/stockprediction/trained/eval/#1539199608843599...
Removing gs://vijays-sandbox/stockprediction/trained/model.ckpt-500.index#1539199665162399...
Removing gs://vijays-sandbox/stockprediction/trained/eval/events.out.tfevents.1539199609.cmle-training-3295948689154592322#1539199742803891...
Removing gs://vijays-sandbox/stockprediction/trained/events.out.tfevents.1539199544.cmle-training-3295948689154592322#1539199750238332...
Removing gs://vijays-sandbox/stockprediction/trained/model.ckpt-600.data-00000-of-00001#1539199687446089...
Removing gs://vijays-sandbox/stockprediction/trained/model.ckpt-500.meta#15391996720216

Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License